# Extracción de actividades del DENUE
> Este tutorial cubre el el uso de los módulos `denue` y `coberturas` para extraer actividades y agregarlas en polígonos 

En este tutorial vamos a usar la librería para extraer algunas actividades específicas y agregarlas por colonias en la CDMX. Las actividades que vamos a extraer son:

* Bancos
* Tiendas de conveniencia
* Restaurantes
* Bares
* Estaciones de policía
* Estaciones de metrobús
* Estaciones de metro

Empezamos por importar todas las librerías que vamos a usar


In [ ]:
import geopandas as gpd
import pandas as pd
from pyLandUseMX.descargas import *
from pyLandUseMX.coberturas import *
from pyLandUseMX.denue import *
from shapely.geometry import Point

In [ ]:
#| echo: false
%load_ext autoreload
%autoreload 2

## Denue
Los bancos, tiendas de conveniencia, restaurantes y bares los vamos a extraer del DENUE

### Bancos

Extraer los bancos es muy fácil, primero nos aseguramos de que estén descargados los datos del DENUE para la Cdmx e instanciamos la clase.

In [ ]:
pth = descarga_denue()
denue = Denue.desde_archivo(pth)
denue.datos.head()

El archivo ya está descargado


,nom_estab,raz_social,codigo_act,nombre_act,per_ocu,tipoCenCom,cve_ent,cve_mun,cve_loc,ageb,manzana,index_right,OBJECTID,Shape_Leng,NOM_CIUDAD,Shape_Le_1,Shape_Area,CVE_SUN,SUN,geometry
0,COMERCIALIZADORA Y DISTRIBUIDORA DE VÍVERES SA...,COMERCIALIZADORA DE VIVERES SA DE CV,115113,Beneficio de productos agrícolas,6 a 10 personas,None,09,017,0001,0884,018,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.06768 19.42209)
1,CRIADERO DE TRUCHAS,None,112512,"Piscicultura y otra acuicultura, excepto camar...",0 a 5 personas,None,09,008,0081,0368,800,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.26444 19.28889)
2,EMPACADORA TEOTIHUACAN,EMPACADORA TEOTIHUACAN SA DE CV,115113,Beneficio de productos agrícolas,0 a 5 personas,None,09,006,0001,0325,020,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.07238 19.40144)
3,FRUTAS CASA GRANDES,FRUTAS CASA GRANDE SPR DE RS DE CV,115113,Beneficio de productos agrícolas,0 a 5 personas,CENTRAL DE ABASTO,09,007,0001,2475,005,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.09481 19.37215)
4,CRIADERO DE TRUCHAS RANCHO LA ROSITA,None,112512,"Piscicultura y otra acuicultura, excepto camar...",0 a 5 personas,None,09,008,0075,0368,800,53,54,630172.981156,Valle de México,630.172981,781912.110166,13,13.0,POINT (-99.27449 19.28433)


Los bancos los podemos identificar a través de la clave SCIAN 522110 (Instituciones de intermediación crediticia y financiera no bursátil:: Banca Múltiple). Vamos a filtrar los bancos y agregar una columna con la categoría para usarla después cuando agreguemos en colonias

In [ ]:
bancos = denue.filtra_scian(['522110'], categoria='Bancos')
bancos.datos[['codigo_act', 'nom_estab', 'Categoria']].head()

,codigo_act,nom_estab,Categoria
577280,522110,BANCO INBURSA AGUILAS AXOMIATLA,Bancos
577281,522110,BANCO SANTANDER,Bancos
577282,522110,BANCO INBURSA BODEGA AURRERA LA VIRGEN,Bancos
577283,522110,BANCO SANTANDER,Bancos
577284,522110,BANCO SANTANDER,Bancos


### Tiendas de conveniencia

Ahora vamos a extraer las tiendas de conveniencia, estas las podemos encontrar en la clave `462112` que corresponde a "Comercio al por menor en minisupers"

In [ ]:
conveniencia = denue.filtra_scian(['462112'], categoria='Conveniencia')
conveniencia.datos[['codigo_act', 'nom_estab', 'Categoria']].head()

,codigo_act,nom_estab,Categoria
220450,462112,ABARROTES Y DULCES EDIMART,Conveniencia
220463,462112,ABARROTES Y MAS,Conveniencia
220467,462112,ABARROTES Y REFRESCOS,Conveniencia
220517,462112,ABARROTES Y VINOS AJUSCO,Conveniencia
220524,462112,ABARROTES YAZMIN,Conveniencia


### Restaurantes

Los restaurantes los podemos identificar por la clave SCIAN `72251` "Servicios de preparación de alimentos y bebidas alcohólicas y no alcohólicas"

In [ ]:
restaurantes = denue.filtra_scian(['72251'], categoria='Restaurantes')
restaurantes.datos[['codigo_act', 'nom_estab', 'Categoria']].head()

,codigo_act,nom_estab,Categoria
725899,722514,ANTOJITOS MEXICANOS,Restaurantes
725900,722513,ANTOJITOS MEXICANOS,Restaurantes
725901,722513,ANTOJITOS MEXICANOS,Restaurantes
725902,722519,ANTOJITOS MEXICANOS,Restaurantes
725903,722513,ANTOJITOS MEXICANOS,Restaurantes


### Bares

Los Bares se identifican con la clave `722412` "Bares, cantinas y similares"

In [ ]:
bares = denue.filtra_scian(['722412'], categoria='Bares')
bares.datos[['codigo_act', 'nom_estab', 'Categoria']].head()

,codigo_act,nom_estab,Categoria
726067,722412,AGUAMIEL Y PULQUES,Bares
726076,722412,AGUA BENDITA,Bares
726087,722412,ADRENALINE WINGS,Bares
726110,722412,ACADEMIA MEXICANA DE SOMMELIERS,Bares
726244,722412,A DIOS LE PIDO RESTAURANTE BAR,Bares


### Estaciones de policía

También es posible encontrar las estaciones de policía en el Denue usando la clave SCIAN `931410` " Impartición de justicia y mantenimiento de la seguridad y el orden público"

In [ ]:
policia = denue.filtra_scian(['931410'], categoria='Policía')
policia.datos[['codigo_act', 'nom_estab', 'Categoria']].head()

,codigo_act,nom_estab,Categoria
988952,931410,MEDICO LEGISTA COORDINACIÓN TERRITORIAL COYOAC...,Policía
988953,931410,MEDICO LEGISTA,Policía
988956,931410,MESA DE DERECHOS HUMANOS DEL CEVASEP 1,Policía
988959,931410,MINISTERIO PUBLICO AGENCIA COYOACAN 3,Policía
988960,931410,MINISTERIO PUBLICO,Policía


### Agregar en colonias

Ya tenemos todos los datos que podemos sacar del Denue, ahora vamos a unirlos en un solo DataFrame y utilizar el módulo `coberturas` para agregarlos en las colonias de la CDMX. Vamos a seleccionar solamente las columnas `Categoria` y `Geometry` para después poder combinar estos datos fácilmente con los que obtengamos de otras fuentes.

In [ ]:
datos_denue = pd.concat([bancos.datos, 
                        conveniencia.datos, 
                        restaurantes.datos, 
                        bares.datos,
                        policia.datos], axis=0)
datos_denue = datos_denue.loc[:, ['Categoria', 'geometry']]
datos_denue.head()

,Categoria,geometry
577280,Bancos,POINT (-99.25366 19.34207)
577281,Bancos,POINT (-99.21235 19.31548)
577282,Bancos,POINT (-99.10697 19.32200)
577283,Bancos,POINT (-99.11336 19.39924)
577284,Bancos,POINT (-99.17190 19.39496)


Ahora ya podemos fácilmente agregar los datos en las colonias, asegurándonos de descargar los datos de las colonias, creando el objeto de la clase `Poligonos` y agregando.

In [ ]:
pth_colonias = descarga_colonias_cdmx()
colonias = Poligonos.desde_archivo(pth_colonias, 'id')
denue_colonia = colonias.agrega_puntos(datos_denue, clasificacion='Categoria')
denue_colonia.datos.head()

,id,Bancos,Bares,Conveniencia,Policía,Restaurantes,nomut,geometry
0,1,0.0,2.0,0.0,0.0,20.0,AGUILERA,"POLYGON ((483311.618 2153254.848, 483332.006 2..."
1,2,0.0,0.0,2.0,0.0,28.0,ALDANA,"POLYGON ((484406.956 2153012.331, 484401.132 2..."
2,3,0.0,0.0,2.0,0.0,68.0,ANGEL ZIMBRON,"POLYGON ((480013.755 2153003.618, 480063.589 2..."
3,4,2.0,0.0,2.0,0.0,74.0,ARENAL,"POLYGON ((484061.118 2152622.284, 484061.678 2..."
4,5,90.0,12.0,6.0,0.0,286.0,CENTRO DE AZCAPOTZALCO,"POLYGON ((480123.402 2154520.928, 480131.648 2..."


## Datos abiertos CDMX

Hasta aquí utilizamos el Denue como fuente de datos, pero también podemos utilizar otras fuentes sin pasar por la clase `Denue`. Por ejemplo, para obtener los datos de estaciones de metro y metrobús, vamos a utilizar los [Datos abiertos de la CDMX](https://datos.cdmx.gob.mx/)

### Estaciones de metro

Las estaciones se pueden descargar de [aquí](https://datos.cdmx.gob.mx/dataset/lineas-y-estaciones-del-metro). Vienen dos _shapes_ uno con las líneas y otro con las estaciones, en este caso vamos a usar sólo el de las estaciones.

In [ ]:
metro = gpd.read_file("../../datos/descargas/STC_Metro_estaciones_utm14n.zip")
metro

,SISTEMA,NOMBRE,LINEA,EST,CVE_EST,CVE_EOD17,TIPO,ALCALDIAS,AÑO,geometry
0,STC Metro,Pantitlán,01,01,STC0101,05014,Terminal / Transbordo,Venustiano Carranza,1984,POINT Z (-99.07474 19.41633 0.00000)
1,STC Metro,Zaragoza,01,02,STC0102,05020,Intermedia,Venustiano Carranza,1969,POINT Z (-99.08229 19.41192 0.00000)
2,STC Metro,Gomez Farías,01,03,STC0103,05007,Intermedia,Venustiano Carranza,1969,POINT Z (-99.09021 19.41648 0.00000)
3,STC Metro,Boulevard Puerto Aéreo,01,04,STC0104,05003,Intermedia,Venustiano Carranza,1969,POINT Z (-99.09626 19.41994 0.00000)
4,STC Metro,Balbuena,01,05,STC0105,05001,Intermedia,Venustiano Carranza,1969,POINT Z (-99.10277 19.42336 0.00000)
...,...,...,...,...,...,...,...,...,...,...
190,STC Metro,Múzquiz,B,17,STCLB17,05185,Intermedia,Estado de México - Ecatepec,2000,POINT Z (-99.04207 19.50149 0.00000)
191,STC Metro,Ecatepec,B,18,STCLB18,05179,Intermedia,Estado de México - Ecatepec,2000,POINT Z (-99.03607 19.51513 0.00000)
192,STC Metro,Olímpica,B,19,STCLB19,05188,Intermedia,Estado de México - Ecatepec,2000,POINT Z (-99.03334 19.52134 0.00000)
193,STC Metro,Plaza Aragón,B,20,STCLB20,05189,Intermedia,Estado de México - Ecatepec,2000,POINT Z (-99.03018 19.52848 0.00000)


Vamos a quedarnos sólo cona la columna de geometría y, para poder combinarlos bien con los que obtuvimos arriba del Denue, agreguemos una columna con la categoría a la que corresponden

In [ ]:
metro.loc[:, 'Categoria'] = "Metro"
metro = metro.loc[:, ['Categoria', 'geometry']]
metro.head()

,Categoria,geometry
0,Metro,POINT Z (-99.07474 19.41633 0.00000)
1,Metro,POINT Z (-99.08229 19.41192 0.00000)
2,Metro,POINT Z (-99.09021 19.41648 0.00000)
3,Metro,POINT Z (-99.09626 19.41994 0.00000)
4,Metro,POINT Z (-99.10277 19.42336 0.00000)


### Metrobús

Ahora hacemos lo mismo con los datos del metrobús que se pueden descargar de [aquí](https://datos.cdmx.gob.mx/dataset/geolocalizacion-metrobus). Estos datos vienen en UTM, entonces vamos a necesitar reproyectarlos para después poderlos concatenar con los otros.

In [ ]:
metrobus = gpd.read_file("../../datos/descargas/Metrobus_estaciones_rutas_utm14n.zip")
metrobus = metrobus.to_crs(4326)
metrobus.loc[:, 'Categoria'] = "Metrobus"
metrobus = metrobus.loc[:, ['Categoria', 'geometry']]
metrobus.head()

,Categoria,geometry
0,Metrobus,POINT (-99.08367 19.43531)
1,Metrobus,POINT (-99.07725 19.42148)
2,Metrobus,POINT (-99.15242 19.44259)
3,Metrobus,POINT (-99.15245 19.44214)
4,Metrobus,POINT (-99.15327 19.43903)


### Agregar en colonias

Igual que con los datos anteriores, vamos a concatenar estas dos bases para agregarlas en las colonias

In [ ]:
transporte = pd.concat([metro, metrobus], axis=0)
transporte_colonias = colonias.agrega_puntos(transporte, clasificacion='Categoria')
transporte_colonias.datos

,id,Metro,Metrobus,nomut,geometry
0,1,0.0,0.0,AGUILERA,"POLYGON ((483311.618 2153254.848, 483332.006 2..."
1,2,0.0,0.0,ALDANA,"POLYGON ((484406.956 2153012.331, 484401.132 2..."
2,3,1.0,0.0,ANGEL ZIMBRON,"POLYGON ((480013.755 2153003.618, 480063.589 2..."
3,4,0.0,0.0,ARENAL,"POLYGON ((484061.118 2152622.284, 484061.678 2..."
4,5,0.0,0.0,CENTRO DE AZCAPOTZALCO,"POLYGON ((480123.402 2154520.928, 480131.648 2..."
...,...,...,...,...,...
1810,1811,0.0,0.0,VILLA PROGRESISTA,"POLYGON ((474406.945 2140353.439, 474418.850 2..."
1811,1812,0.0,0.0,VILLA SOLIDARIDAD,"POLYGON ((477048.978 2143204.554, 477054.695 2..."
1812,1813,0.0,0.0,VILLA VERDUN,"POLYGON ((473081.154 2138733.281, 473070.790 2..."
1813,1814,0.0,0.0,ZENON DELGADO,"POLYGON ((477329.014 2144260.214, 477335.725 2..."


## Unir datos y exportar

Ya tenemos las bases agregadas pogracias por recordarme jejeje
r colonias que extrajimos de dos fuentes diferentes: el Denue y los Datos Abiertos de la CDMX. Ahora vamos a unir ambas bases y exportar los datos como _shapefile_.

El método `Poligonos.agrega_puntos` siempre nos regresa los polígonos originales completos, entonces unirlos es sencillo

In [ ]:
completo = transporte_colonias.datos.merge(denue_colonia.datos[['id','Bancos', 'Bares', 'Conveniencia', 'Policía', 'Restaurantes']], on='id')
completo

,id,Metro,Metrobus,nomut,geometry,Bancos,Bares,Conveniencia,Policía,Restaurantes
0,1,0.0,0.0,AGUILERA,"POLYGON ((483311.618 2153254.848, 483332.006 2...",0.0,2.0,0.0,0.0,20.0
1,2,0.0,0.0,ALDANA,"POLYGON ((484406.956 2153012.331, 484401.132 2...",0.0,0.0,2.0,0.0,28.0
2,3,1.0,0.0,ANGEL ZIMBRON,"POLYGON ((480013.755 2153003.618, 480063.589 2...",0.0,0.0,2.0,0.0,68.0
3,4,0.0,0.0,ARENAL,"POLYGON ((484061.118 2152622.284, 484061.678 2...",2.0,0.0,2.0,0.0,74.0
4,5,0.0,0.0,CENTRO DE AZCAPOTZALCO,"POLYGON ((480123.402 2154520.928, 480131.648 2...",90.0,12.0,6.0,0.0,286.0
...,...,...,...,...,...,...,...,...,...,...
1810,1811,0.0,0.0,VILLA PROGRESISTA,"POLYGON ((474406.945 2140353.439, 474418.850 2...",2.0,0.0,0.0,0.0,6.0
1811,1812,0.0,0.0,VILLA SOLIDARIDAD,"POLYGON ((477048.978 2143204.554, 477054.695 2...",0.0,0.0,0.0,0.0,0.0
1812,1813,0.0,0.0,VILLA VERDUN,"POLYGON ((473081.154 2138733.281, 473070.790 2...",0.0,0.0,0.0,2.0,2.0
1813,1814,0.0,0.0,ZENON DELGADO,"POLYGON ((477329.014 2144260.214, 477335.725 2...",0.0,0.0,0.0,0.0,14.0


In [ ]:
completo.to_file("../../datos/descargas/actividades_cdmx.shp")

/tmp/ipykernel_14136/2170292792.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  completo.to_file("../../datos/descargas/actividades_cdmx.shp")
